In [9]:
# from torchvision.models import resnet50
# model = resnet50(weights="IMAGENET1K_V1")

In [61]:
%load_ext autoreload
%autoreload 2


import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image

import pandas as pd


import psutil
import os
from timeit import default_timer as timer

from utilities import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
                                )

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [14]:
net = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Using cache found in /home/mike/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


Train

In [ ]:
# for epoch in range(1):  # loop over the dataset multiple times

#     running_loss = 0.0
#     for i, data in enumerate(trainloader, 0):
#         # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         outputs = net(inputs)
#         loss = loss_fn(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()
#         if i % 100 == 99:    # print every 2000 mini-batches
#             print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
#             running_loss = 0.0

# print('Finished Training')

Test

In [74]:
report = pd.DataFrame(columns=['batch_size', 'cpu_util', 'memory_util', 'time_taken'])

In [75]:
#  some codde here....
end_time = timer()

for i in range(16):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    net.eval()

    batch_size = 2 ** i

    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

    start_time = timer()
    psutil.cpu_percent(0) # Mark cpu utilization

    with torch.inference_mode():
        for idx, data in enumerate(testloader):
            # print('The CPU usage is: ', psutil.cpu_percent(0))
            
            images, labels = data
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if idx == 0:
                mem = psutil.virtual_memory()[3]/1000000000

    end_time = timer()
    print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

    cpu = psutil.cpu_percent(0)
    print('The CPU usage is: ', cpu)

    report = pd.concat([report, pd.DataFrame({'batch_size': 2 ** i,
                                    'cpu_util' : cpu, 
                                    'memory_util':mem, 
                                    'time_taken' :end_time - start_time},index=[i])])

     # Report CPU since last mark
    report

Files already downloaded and verified
Files already downloaded and verified
Accuracy of the network on the 10000 test images: 81 %
The CPU usage is:  66.2


/tmp/ipykernel_94773/1374572488.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  report = pd.concat([report, pd.DataFrame({'batch_size': 2 ** i,


Files already downloaded and verified
Files already downloaded and verified
Accuracy of the network on the 10000 test images: 81 %
The CPU usage is:  67.5
Files already downloaded and verified
Files already downloaded and verified
Accuracy of the network on the 10000 test images: 81 %
The CPU usage is:  67.3
Files already downloaded and verified
Files already downloaded and verified
Accuracy of the network on the 10000 test images: 81 %
The CPU usage is:  67.9
Files already downloaded and verified
Files already downloaded and verified


KeyboardInterrupt: 

In [76]:
report

,batch_size,cpu_util,memory_util,time_taken
0,1,66.2,3.827519,80.805330
1,2,67.5,3.694072,48.877474
2,4,67.3,3.733320,33.634100
3,8,67.9,3.678212,25.829052
